# Tests Nov 4

In [20]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [40]:
from invesscience.joanna_merge import get_training_data 

In [41]:
companies = get_training_data(reference="a")

In [42]:
companies.head()

,id,normalized_name,category_code,founded_at,description,closed_at,exit,exit_date,country_code,state_code,...,top_20,top_50,top_5_bool,top_20_bool,top_50_bool,n_female_founders,female_ratio,mean_comp_founded_ever,mean_comp_founded_before,target
0,c:1349,move networks,software,NaT,software Move Networks is a streaming tech pla...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,c:4584,modern feed,games_video,2007-01-01,games_video [Modern Feed](http://www.moder...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
2,c:16612,micron technology,semiconductor,1978-01-01,semiconductor Semiconductor Applications Mic...,NaN,ipo,1990-01-02 00:00:00,USA,ID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,c:28,cisco,network_hosting,1984-01-01,network_hosting Cisco designs and sells ha...,NaN,ipo,1990-02-23 00:00:00,USA,CA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,c:29,yahoo,web,1994-01-01,web News and On-line directory Yahoo was fou...,NaN,ipo,1996-04-12 00:00:00,USA,CA,...,0.0,0.2,1.0,0.0,1.0,0.0,0.0,1.2,0.0,1


In [43]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3241 entries, 0 to 3240
Data columns (total 39 columns):
id                           3241 non-null object
normalized_name              3241 non-null object
category_code                3115 non-null object
founded_at                   2642 non-null datetime64[ns]
description                  3239 non-null object
closed_at                    360 non-null object
exit                         3241 non-null object
exit_date                    797 non-null object
country_code                 3118 non-null object
state_code                   2241 non-null object
date_series_a                3241 non-null datetime64[ns]
participants_a               3241 non-null int64
raised_amount_usd_a          3241 non-null float64
timediff_founded_series_a    2642 non-null float64
rounds_before_a              3241 non-null float64
raised_before_a              3165 non-null float64
participants_before_a        3241 non-null float64
phd                      

In [37]:
companies[companies.rounds_before_a.isnull()]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3241 entries, 0 to 3240
Data columns (total 39 columns):
id                           3241 non-null object
normalized_name              3241 non-null object
category_code                3115 non-null object
founded_at                   2642 non-null datetime64[ns]
description                  3239 non-null object
closed_at                    360 non-null object
exit                         3241 non-null object
exit_date                    797 non-null object
country_code                 3118 non-null object
state_code                   2241 non-null object
date_series_a                3241 non-null datetime64[ns]
participants_a               3241 non-null int64
raised_amount_usd_a          3241 non-null float64
timediff_founded_series_a    2642 non-null float64
rounds_before_a              399 non-null float64
raised_before_a              323 non-null float64
participants_before_a        323 non-null float64
phd                         

In [7]:
companies.mean_comp_founded_before.value_counts()

0.000000    3078
1.000000      79
0.500000      71
0.333333      44
0.250000      17
0.200000       9
2.000000       8
0.666667       5
1.500000       3
0.400000       2
0.800000       2
0.750000       1
4.000000       1
3.000000       1
0.600000       1
1.333333       1
0.571429       1
1.666667       1
0.375000       1
Name: mean_comp_founded_before, dtype: int64

In [10]:
exits ={"acquisition":1, "ipo":1, "no exit":0}
companies["target"] = companies.exit.map(exits)

In [26]:
companies.category_code.value_counts()

software            1740
web                 1056
biotech              669
mobile               481
games_video          446
enterprise           389
cleantech            379
advertising          355
hardware             304
semiconductor        284
ecommerce            281
public_relations     268
network_hosting      243
other                196
security             132
search               129
consulting            94
medical               73
analytics             69
finance               52
social                36
music                 31
news                  28
education             28
manufacturing         27
messaging             27
travel                20
health                19
transportation        12
automotive            12
nanotech              11
photo_video           10
real_estate           10
legal                 10
sports                 9
fashion                8
hospitality            7
nonprofit              4
local                  2
government             1
